# DataSens E1_v1 — 03_ingest_sources

- Objectifs: insérer données simulées (seed) pour tests
- Prérequis: 02_schema_create
- Sortie: lignes de démonstration dans `document`
- Guide: docs/GUIDE_TECHNIQUE_E1.md



> Notes:
> - Cette étape peuple la base avec un échantillon minimal pour tester CRUD.
> - `executescript` exécute plusieurs INSERT d’un coup (référentiels + un document).
> - Le `document` est rattaché à un `flux` pour la traçabilité.
> - L’opération (`DB_SEED`) est inscrite dans le fichier de versioning.


In [ ]:
# DataSens E1_v1 - 03_ingest_sources
# 🌱 Bootstrap: données de test (repris du monolithe)
import sqlite3
from datetime import datetime
from pathlib import Path

# Détection robuste du dossier projet (remonte jusqu'à trouver le dossier avec notebooks/ et docs/)
current = Path.cwd()
PROJECT_ROOT = None
# Remonter jusqu'à trouver un dossier qui contient à la fois notebooks/ et docs/
while current != current.parent:
    if (current / "notebooks").exists() and (current / "docs").exists():
        PROJECT_ROOT = current
        break
    current = current.parent
else:
    # Fallback: utilise le répertoire courant si pas trouvé
    PROJECT_ROOT = Path.cwd()

DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
print(f"📂 Racine projet détectée : {PROJECT_ROOT}")

# Vérifier que la base existe et a des tables
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Vérifier si les tables existent, sinon créer le minimum
cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='type_donnee'")
if not cur.fetchone():
    print("⚠️ Les tables n'existent pas. Exécutez d'abord 02_schema_create.ipynb")
    print(f"   Base trouvée: {DB_PATH}")
    conn.close()
    raise Exception("Schéma manquant: exécutez 02_schema_create.ipynb avant 03_ingest_sources.ipynb")

cur.executescript("""
INSERT INTO type_donnee (libelle, description, frequence_maj, categorie_metier)
VALUES ('Fichier','CSV Kaggle','hebdomadaire','descriptive');

INSERT INTO source (id_type_donnee, nom, url, fiabilite)
VALUES (1,'Kaggle CSV – FR textes','https://kaggle.com/xxx',0.8);

INSERT INTO flux (id_source, date_collecte, format, manifest_uri)
VALUES (1,'2025-10-27T10:00:00','csv','data/raw/kaggle/sample.csv');

INSERT INTO document (id_flux, titre, texte, langue, date_publication)
VALUES (1,'JO 2024 : fierté nationale','Les JO ont boosté le moral','fr','2024-07-27');
""")

conn.commit()

# 📊 Visualisation : Afficher les données insérées avec pandas
import pandas as pd
import matplotlib.pyplot as plt

# 📋 Tables de données réelles : Afficher toutes les données insérées

# Table 1 : Type de donnée
print("\n📊 Table 'type_donnee' :")
df_type_donnee = pd.read_sql_query("SELECT * FROM type_donnee", conn)
display(df_type_donnee)

# Table 2 : Source
print("\n📊 Table 'source' :")
df_source = pd.read_sql_query("SELECT * FROM source", conn)
display(df_source)

# Table 3 : Flux
print("\n📊 Table 'flux' :")
df_flux = pd.read_sql_query("SELECT * FROM flux", conn)
display(df_flux)

# Table 4 : Document (avec jointures pour contexte)
print("\n📊 Table 'document' (avec contexte) :")
df_docs = pd.read_sql_query("""
    SELECT d.id_doc, d.titre, d.texte, d.langue, d.date_publication,
           s.nom AS source, f.format, f.date_collecte
    FROM document d
    JOIN flux f ON d.id_flux = f.id_flux
    JOIN source s ON f.id_source = s.id_source
    ORDER BY d.id_doc
""", conn)
display(df_docs)

# Graphique : Répartition par langue (si plusieurs)
if len(df_docs) > 0:
    if len(df_docs['langue'].unique()) > 1:
        lang_counts = df_docs['langue'].value_counts()
        plt.figure(figsize=(8, 6))
        colors = plt.cm.Set2(range(len(lang_counts)))
        plt.pie(lang_counts.values, labels=lang_counts.index, autopct='%1.1f%%', 
                colors=colors, startangle=90)
        plt.title("📊 Répartition des documents par langue", fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    else:
        print(f"   ✅ Tous les documents sont en '{df_docs['langue'].iloc[0]}'")

# Statistiques de la chaîne de traçabilité
stats = {
    "Types de données": len(pd.read_sql_query("SELECT COUNT(*) as c FROM type_donnee", conn).iloc[0]),
    "Sources": len(pd.read_sql_query("SELECT COUNT(*) as c FROM source", conn).iloc[0]),
    "Flux": len(pd.read_sql_query("SELECT COUNT(*) as c FROM flux", conn).iloc[0]),
    "Documents": len(df_docs)
}

print("\n📈 Chaîne de traçabilité complète :")
for label, count in stats.items():
    print(f"   • {label} : {count}")

# Visualisation de la chaîne
if stats["Documents"] > 0:
    plt.figure(figsize=(10, 5))
    bars = plt.bar(stats.keys(), stats.values(), color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    for bar, value in zip(bars, stats.values()):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                str(value), ha='center', va='bottom', fontweight='bold')
    plt.title("📊 Chaîne de traçabilité : type_donnee → source → flux → document", 
              fontsize=12, fontweight='bold')
    plt.ylabel("Nombre d'éléments", fontsize=11)
    plt.grid(axis="y", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()

with open(PROJECT_ROOT / "README_VERSIONNING.md", "a", encoding="utf-8") as f:
    f.write(f"- **{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}** | `DB_SEED` | Insertion du jeu de données minimal\n")
print("\n✅ Données de test insérées et visualisées.")


📂 Racine projet détectée : c:\Users\Utilisateur\Desktop\DataSens\notebooks\datasens_E1_v1
🌱 Données de test insérées.
